# Homework 3 - places of the world

**Name**: Giulia Iadisernia || **Matricola**: 2065450 || **Email**: iadisernia.2065450@studenti.uniroma1.it <br>
**Name**: Stefano La Commare|| **Matricola**: 1919033 || **Email**: lacommare.1919033@studenti.uniroma1.it <br>
**Name**: Francesco Ferrusi || **Matricola**: 2081332 || **Email**: francesco.ferrusi@outlook.it

## 1 Data Collection

### 1.1. Get the list of places

####  Import modules

In [21]:
import warnings
warnings.filterwarnings('ignore')
import requests
from bs4 import BeautifulSoup, Tag, NavigableString
import os
import pandas as pd
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
import spacy
nlp=spacy.load('en_core_web_sm')
from collections import Counter
from functools import reduce
from itertools import islice
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import operator
from datetime import datetime
import folium
import math

#### Get list of pages' URLs

* Every page's URL looks like `https://www.atlasobscura.com/places?page=i&sort=likes_count` for $1$ $\leq$ $i$ $\leq$ $400$
* Therefore, we obtained a list of links (URLs) called `list_pages_url`

In [275]:
list_pages_url = []

for i in range(1, 401):
    if i == 1:
        URL = 'https://www.atlasobscura.com/places?sort=likes_count'
    else:
        URL = 'https://www.atlasobscura.com/places?page=' + str(i) + '&sort=likes_count'
    list_pages_url.append(URL)

In [276]:
len(list_pages_url)

400

#### Get list of places for every page

* The URLs are composed by `https://www.atlasobscura.com` $+$ `the name of of the place` $+$ `?page=i&sort=likes_count` $for$ $1$ $\leq$ $i$ $\leq$ $400$.
* We created a list containing all of the links named `complete_url_list`.

In [168]:
complete_url_list = []

for i in range(0, len(list_pages_url)):
    page = list_pages_url[i]
    r = requests.get(page)
    soup = BeautifulSoup(r.text, 'html.parser')
    for a in soup.find_all('a', attrs = {'class': "content-card content-card-place"}):
        if i+1 == 1: # first link
            l = 'https://www.atlasobscura.com' + a['href'] + '?sort=likes_count'
        else:
            l = 'https://www.atlasobscura.com' + a['href'] + '?page=' + str(i+1) + '&sort=likes_count'
        complete_url_list.append(l)

In [170]:
len(complete_url_list) #7200

7200

* Finally, we create an `output.txt` file that contains all of the links in the `complete_url_list`

In [188]:
# open file
with open('output.txt', 'w+') as f:
     
    for items in complete_url_list:
        f.write('%s\n' %items)
     
    print("File written successfully")
    
f.close()

File written successfully


### 1.2. Crawl places

* The function `save_links_as_html_files(txt_file, directory)` takes as inputs the `.txt` file containing the list of URLs, and the `directory` where we want to save our files.
* It creates 400 folders named `Page_i` for $1$ $\leq$ $i$ $\leq$ $400$, where each folder contains 18 `.html` files named `Place_j.html` for for $1$ $\leq$ $j$ $\leq$ $7200$.

In [715]:
def save_links_as_html_files(txt_file, directory):
    
    with open(txt_file) as f:
        files = [line.rstrip('\n') for line in f]
    f.close()
    
    folder_name = ""
    folder_dir  = ""
    file_name   = ""
    file_dir    = ""
    page_num    = 1

    for m in range(0, 1800, 18):

        folder_name = "Page_" + str(page_num)
        folder_dir = directory + "\\" + folder_name

        # create folder directory if it does not exist already
        if not os.path.exists(folder_dir):
            os.makedirs(folder_dir)

        for i in range(0, 18):

            file = files[i+m]
            r = requests.get(file)

            file_name = "Place_" + str(i+m+1) + ".html"
            file_dir = folder_dir + "\\" + file_name

            with open(file_dir, "wb+") as f:
                f.write(r.content)
                
            f.close()
            
        page_num += 1
        

In [716]:
save_links_as_html_files('output.txt', 'C:/Users\iadig\Desktop\HW3_html_files')

### 1.3 Parse downloaded pages

* The function `parse_html_files(directory)` takes as input the `directory` of the folders containing the `.html` files outputted in point $1.2$.
* It returns a list of parsed `.html` files. We used `BeautifulSoup()` to do so.

In [504]:
def parse_html_files(directory):

    folder_name = ""
    folder_dir  = ""
    file_name   = ""
    file_dir    = ""
    page_num    = 1
    List = []

    for m in range(0, 1800, 18):

        folder_name = "Page_" + str(page_num)
        folder_dir = directory + "\\" + folder_name

        for i in range(0, 18):

            file_name = "Place_" + str(i+m+1) + ".html"
            file_dir = folder_dir + "\\" + file_name

            with open(file_dir, "rb") as f:
                soup = BeautifulSoup(f.read(), 'html.parser')

            f.close()
              
            List.append(soup)

        page_num += 1
    
    return List

In [505]:
soup_list = parse_html_files('C:/Users\iadig\Desktop\HW3_html_files')

### The functions

#### 1) Place Name (to save as placeName): String.

In [594]:
def find_placeName(soup_list):
    
    placeName_list = []
    
    for soup in soup_list:
        
        title = soup.find_all('h1', {'class':'DDPage__header-title'})
        
        if len(title) > 0: # there is a title
            placeName = str(title[0].contents[0])
            placeName_list.append(placeName)
            
        else:
            placeName_list.append(float('NaN'))
    
    return placeName_list

#### 2) Place Tags (to save as placeTags): List of Strings.

In [593]:
def find_placeTags(soup_list):
    
    placeTags_list = []
    
    for soup in soup_list:
        
        location = soup.find_all("div", {'class': 'DDPage__header-place-location'})
        
        if len(location) > 0: # there is a title
            loc = location[0].contents[0]
            if type(loc) == Tag:
                placeTags = str(loc.contents[0])
            else:
                placeTags = str(loc)
                
            placeTags_list.append(placeTags)
            
        else:
            placeTags_list.append(float('NaN'))
    
    return placeTags_list


#### 3) No. of people who have been there (to save as numPeopleVisited): Integer.

In [592]:
def find_numPeopleVisited(soup_list):
    
    numPeopleVisited_list = []
    
    for soup in soup_list:
        
        num = soup.find_all('div', {'class': 'title-md item-action-count'})
        
        if len(num) > 0: # there is a title
            numPeopleVisited = int(str(num[0].contents[0]))
            numPeopleVisited_list.append(numPeopleVisited)
            
        else:
            numPeopleVisited_list.append(float('NaN'))
    
    return numPeopleVisited_list

#### 4) No. of people who want to visit the place(to save as numPeopleWant): Integer.

In [591]:
def find_numPeopleWant(soup_list):
    
    numPeopleWant_list = []
    
    for soup in soup_list:
        
        num = soup.find_all('div', {'class': 'title-md item-action-count'})
        
        if len(num) > 0: # there is a title
            numPeopleWant = int(str(num[1].contents[0]))
            numPeopleWant_list.append(numPeopleWant)
            
        else:
            numPeopleWant_list.append(float('NaN'))
    
    return numPeopleWant_list

#### 5) Description (to save as placeDesc): String. Everything from under the first image up to "know before you go" (orange frame on the example image).

In [523]:
def find_placeDesc(soup_list):
    
    placeDesc_list = []
    
    for soup in soup_list:
        
        long_description = [x.text.strip() for x in soup.find_all('div', {'id': 'place-body'})]
        
        if len(long_description) > 0:
            
            placeDesc_list.append(long_description[0])
            
        else:
            
            placeDesc_list.append(float('NaN'))
    
    return placeDesc_list

#### 6) Short Description (to save as placeShortDesc): String. Everything from the title and location up to the image (blue frame on the example image).

In [590]:
def find_placeShortDesc(soup_list):
    
    placeShortDesc_list = []
    
    for soup in soup_list:
        
        meta = soup.find_all('meta', {'property': "og:description"})
        
        if len(meta) > 0:
            short_description = meta[0]['content'].rstrip()
            
            placeShortDesc_list.append(short_description)
            
        else:
            placeShortDesc_list.append(float('NaN'))
        

    return placeShortDesc_list

#### 7) Nearby Places (to save as placeNearby): Extract the names of all nearby places, but only keep unique values: List of Strings. 

In [450]:
def find_placeNearby(soup_list):
    
    placeNearby_list = []
    
    for soup in soup_list:
        
        placeNearby = []
        
        div = soup.find_all('div', class_="DDPageSiderailRecirc__item-title")
        
        for i in range(0, len(div)):
            placeNearby.append(str(div[i].contents[0]))
            
        if len(placeNearby) > 0:
            placeNearby_list.append(placeNearby)
        else:
            placeNearby_list.append(float('NaN'))

    return placeNearby_list

#### 8) Address of the place(to save as placeAddress): String.

In [595]:
def find_placeAdress(soup_list):
    
    placeAdress_list = []
    
    for soup in soup_list:
        
        address = soup.find_all('address', {'class':"DDPageSiderail__address"})
        
        if len(address) > 0:
        
            placeAdress = ""
            for el in address[0].find_all('div')[0].contents:
                if type(el) == NavigableString:
                    if placeAdress == "":
                        placeAdress = el
                    else:
                        placeAdress = placeAdress + ' ' + el

            placeAdress = placeAdress.rstrip("\n")
            
            placeAdress_list.append(placeAdress)
            
        else:
            
            placeAdress_list.append(float('NaN'))

        
    return placeAdress_list

#### 9) Altitude and Longitude of the place's location(to save as placeAlt and placeLong): Integers

In [454]:
def find_placeAlt_placeLong(soup_list):
    
    placeAlt_list = []
    placeLong_list = []
    
    for soup in soup_list:
    
        div = soup.find_all("div", {'class': 'DDPageSiderail__coordinates js-copy-coordinates'})

        if len(div) > 0:
            coordinates = div[0]['data-coordinates']

            placeAlt, placeLong = coordinates.split(", ")

            placeAlt_list.append(float(placeAlt))
            placeLong_list.append(float(placeLong))

        else:
            placeAlt_list.append(float('NaN'))
            placeLong_list.append(float('NaN'))
        
    return [placeAlt_list, placeLong_list]

#### 10) The username of the post editors (to save as placeEditors): List of Strings.

In [581]:
def find_placeEditors(soup_list):
    
    placeEditors_list = []
    
    for soup in soup_list:
    
        placeEditors = []
        
        contributors = soup.find_all("div", {'class': 'DDPContributorsList'})

        for el in contributors:
            tag = el.find_all("span")
                
        if len(tag) > 0:
                    
            for i in range(len(tag)):
                placeEditors.append(str(tag[i].contents[0]))
                
            placeEditors_list.append(placeEditors)
                
        elif len(tag) == 0:
                    
            placeEditors_list.append(float('NaN'))
            
    return placeEditors_list

#### 11) Post publishing date (to save as placePubDate): datetime.

In [596]:
def find_placePubDate(soup_list):
    
    placePubDate_list = []
    
    for soup in soup_list:
        
        div = soup.find_all("div", {'class': 'DDPContributor__name'})
        
        if len(div) > 0:
            
            placePubDate = str(div[0].contents[0])
            placePubDate_list.append(placePubDate)
                            
        else:
            
            placePubDate_list.append(float('NaN'))
        
    return placePubDate_list

#### 12) The names of the lists that the place was included in (to save as placeRelatedLists): List of Strings.

In [586]:
def find_placeRelatedLists(soup_list):
    
    placeRelated_list = []
    
    for soup in soup_list:
        
        placeRelatedLists = []
        
        external_div = soup.find_all('div', class_="card-grid CardRecircSection__card-grid js-inject-gtm-data-in-child-links")
        
        if len(external_div) > 2:
            
            internal_h3 = external_div[2].find_all('h3', class_="Card__heading --content-card-v2-title js-title-content")
            
            for place_list in internal_h3:
                placeRelatedLists.append(str(place_list.find_all("span")[0].contents[0]))
                 
            placeRelated_list.append(placeRelatedLists)
                
        else:
            placeRelated_list.append(float('NaN'))
            

    return placeRelated_list

#### 13) The names of the related places (to save as placeRelatedPlaces): List of Strings.

In [615]:
def find_placeRelatedPlaces(soup_list):
    
    placeRelatedPlaces_list = []
    
    for soup in soup_list:
        
        placeRelatedPlaces = []
        
        external_div = soup.find_all('div', class_="card-grid CardRecircSection__card-grid js-inject-gtm-data-in-child-links")
        
        if len(external_div) > 1:
            
            internal_h3 = external_div[1].find_all('h3', class_="Card__heading --content-card-v2-title js-title-content")
            
            for place_list in internal_h3:
                placeRelatedPlaces.append(str(place_list.find_all("span")[0].contents[0]))
                
            placeRelatedPlaces_list.append(placeRelatedPlaces)
                
        else:
            placeRelatedPlaces_list.append(float('NaN'))
            

    return placeRelatedPlaces_list

#### 14) The URL of the page of the place (to save as placeURL):String

In [2]:
def find_placeURL(soup_list):
    
    placeURL_list = []
    
    for soup in soup_list:
        
        link = soup.find_all("link", {'rel': 'canonical'})
        
        if len(link) > 0:
            
            placeURL = str(link[0]['href'])
            
            placeURL_list.append(placeURL)
            
        else:
            
            placeURL_list.append(float('NaN'))
        
    return placeURL_list

### Dataset Creation

* Now, we call the `functions` to extract the places' information.
* If the some of the information that we want is not found, we simply insert a `NaN` values instead.

In [602]:
placeName_list = find_placeName(soup_list)
placeTags_list = find_placeTags(soup_list)
numPeopleVisited_list = find_numPeopleVisited(soup_list)
numPeopleWant_list = find_numPeopleWant(soup_list)
placeDesc_list = find_placeDesc(soup_list)
placeEditors_list = find_placeEditors(soup_list)
placeRelated_list = find_placeRelatedLists(soup_list)
placeRelatedPlaces_list = find_placeRelatedPlaces(soup_list)
placeShortDesc_list = find_placeShortDesc(soup_list)
placeNearby_list = find_placeNearby(soup_list)
placeAdress_list = find_placeAdress(soup_list)
placeAlt_list = find_placeAlt_placeLong(soup_list)[0]
placeLong_list = find_placeAlt_placeLong(soup_list)[1]
placePubDate_list = find_placePubDate(soup_list)
placeRelated_list = find_placeRelatedLists(soup_list)
placeURL_list = find_placeURL(soup_list)

In [ ]:
# create the dataframe

df = pd.DataFrame({'placeName':placeName_list, 'placeTags':placeTags_list, 'numPeopleVisited':numPeopleVisited_list,
                  'numPeopleWant':numPeopleWant_list, 'placeDesc':placeDesc_list, 'placeShortDesc':placeShortDesc_list,
                  'placeNearby':placeNearby_list, 'placeAdress':placeAdress_list, 'placeAlt':placeAlt_list,
                   'placeLong':placeLong_list, 'placeEditors':placeEditors_list, 'placePubDate': placePubDate_list,
                  'placeRelated':placeRelated_list, 'placeRelatedPlaces':placeRelatedPlaces_list, 'placeURL':placeURL_list})

* Finally, we save the `df` as a `.tsv` file.

In [8]:
df.to_csv('C:/Users\iadig\Desktop\hw3_tsv\complete_places.tsv', sep = '\t', index = False)

## 2. Search Engine

In [13]:
webdata = pd.read_csv(r"C:/Users\stefa\Desktop\complete_places.tsv", sep='\t')

### Pre-processing

* Now we want to remove stopwords, remove punctuation and stem the long description of each place.
* We can find stopwords with ntlk package and punctuation with string package. For the stemming we use SnowballStemmer from ntlk

In [14]:
stemmer = SnowballStemmer("english")
stop_words = set(stopwords.words('english'))
punctuations = string.punctuation + "’"

* So we create a function that given the data and the name of a variable, will give a new column in the dataset with the pre-processing operations mentioned above.

In [15]:
def pre_proces(data,name_column):
    var = data[name_column]
    data['list_words_'+name_column] = var.apply(lambda x: ' '.join([w for w in x.split() if w.lower() not in (stop_words)]))
    data['list_words_'+name_column] = data['list_words_'+name_column].apply(lambda x: ' '.join(([(w.translate(str.maketrans('', '', punctuations))) for w in x.split()])))
    data['list_words_'+name_column] = data['list_words_'+name_column].apply(lambda x: [stemmer.stem(y) for y in x.split()])
    return data['list_words_'+name_column]

* Pre-processing placeDesc and obtaining list_words_placeDesc.

In [18]:
pre_proces(webdata,"placeDesc")

0       [first, new, york, citi, subway, built, oper, ...
1       [open, 1839, highgat, one, london, infam, ceme...
2       [ornat, 19thcenturi, paint, roof, cobbl, floor...
3       [locat, jetti, san, francisco, bay, wave, orga...
4       [2004, parisian, polic, assign, train, exercis...
                              ...                        
7195    [villag, north, carolina, outer, bank, road, s...
7196    [eli, cathedr, date, far, back, seventh, centu...
7197    [eeri, overrun, lush, greeneri, abandon, theme...
7198    [washington, dcs, residenti, neighborhood, typ...
7199    [towner, drive, along, dull, stretch, road, ta...
Name: list_words_placeDesc, Length: 7200, dtype: object

## 2.1.1) Create your index!

### Vocabulary

* Now we want to create a vocabulary that links every word in all the long description to an ID.

In [ ]:
uniqueWord = [];
for i in range(webdata.shape[0]):
    for j in webdata.list_words_placeDesc[i]:
        if j not in uniqueWord:
            uniqueWord.append(j)
vocabulary={}
index_id=[]
for i in range(len(uniqueWord)):
    vocabulary[uniqueWord[i]]=i

### Inverted index

* We create the Inverted index that is a dictionary that gives us, for each word ID, the documents in which it is contained.

In [22]:
inverted_idx = {}
for word, item_id in tqdm(vocabulary.items()):
    inverted_idx[item_id] = list(webdata[webdata.list_words_placeDesc.apply(lambda row: word in row)].index)

100%|██████████| 58793/58793 [17:05<00:00, 57.34it/s]


## 2.1.2) Execute the query

* This is a function to pre-processing easily the queries input in the following functions.

In [33]:
def pre_proces_query(query):
    stem_query = ' '.join(([(w.translate(str.maketrans('', '', punctuations))) for w in query.split()]))
    stem_query = ' '.join([w for w in stem_query.split() if w.lower() not in (stop_words)])
    stem_query = [stemmer.stem(y) for y in stem_query.split()]
    return stem_query

In [34]:
def inverted_index(query):
    quer = pre_proces_query(query)
    results =[set(inverted_idx[item]) for item in [vocabulary[word] for word in quer]]
    first = results[0]
    for other in results[1:]:
        first = first.intersection(other)
    docID = list(reduce(lambda x,y: x.intersection(y), results))
    return webdata.loc[docID][["placeName","placeDesc","placeURL"]]

* Query example

In [162]:
query_str='new york'

In [165]:
inverted_index(query_str)

,placeName,placeDesc,placeURL
0,City Hall Station,The first New York City subway was built and o...,https://www.atlasobscura.com/places/city-hall-...
2053,The Berkeley Pit,Copper from this former open-pit mine helped t...,https://www.atlasobscura.com/places/berkeley-pit
4106,The American Geographical Society Library,Within the campus of the University of Wiscons...,https://www.atlasobscura.com/places/the-americ...
6156,The Girl in Blue,"It was Christmas Eve in 1933 in Willoughby, Oh...",https://www.atlasobscura.com/places/the-girl-i...
4108,Site of New York Slave Market,"Now an anonymous condominium, the building sta...",https://www.atlasobscura.com/places/site-of-ne...
...,...,...,...
2013,Riot Grrrl Collection at NYU,The Fales Library at New York University has m...,https://www.atlasobscura.com/places/riot-grrrl...
4061,The Sugar House Prison Window,Hidden away on a wall of the New York City Pol...,https://www.atlasobscura.com/places/the-sugar-...
2030,Ruins of the Cornish Estate,One of the great things to do in New York City...,https://www.atlasobscura.com/places/ruins-of-t...
6132,Rokhat Kosher Bakery,"In this big, diverse, and sometimes contentiou...",https://www.atlasobscura.com/places/rokhat-kos...


## 2.2.1) Inverted index

* Now we create a new Inverted index that for each word it gives us the list of documents in which it is contained and the relative tfIdf score

In [204]:
tfidf= TfidfVectorizer(input='content', lowercase=False , tokenizer=lambda text:text, max_df=0.5, min_df=2)
result=tfidf.fit_transform(webdata.list_words_placeDesc)
result_dense= result.todense()
results_dense_list =result_dense.tolist()
df1 = pd.DataFrame(results_dense_list, index=webdata.index, columns=tfidf.get_feature_names_out())

* df1 represents the sparse matrix of tfidf scores between words and place descriptions

In [188]:
di = {}
for word in df1:
    li = []
    for i,tfidf in enumerate(df1[word]):
        if tfidf!=0:
            li.append((i,tfidf))
    di[vocabulary[word]] = li

* Now for example the word "pizza" has ID: 11056

In [192]:
print(vocabulary["pizza"])

11056


In [193]:
print(di[11056])

[(371, 0.08691611638664178), (1302, 0.6564147166224242), (1566, 0.049566071487348276), (1648, 0.2051938131032298), (1984, 0.04641030958891165), (1990, 0.07830609468186255), (2043, 0.8679664150183114), (3091, 0.5041686987683972), (3414, 0.06449836499339344), (3480, 0.4537086228766503), (3835, 0.05350701960727532), (3928, 0.07474773850152143), (3945, 0.07705936402997982), (4139, 0.21527139266937365), (4178, 0.043768216626096826), (4526, 0.6009112549476118), (4736, 0.6209293048283294), (4781, 0.19921095652863355), (5854, 0.19180935534391108), (5995, 0.08421189927177535), (6175, 0.4489333629137928), (6205, 0.11802056880216781), (6259, 0.32465652454239824), (6366, 0.1586339946479676), (6396, 0.25595281311528567), (6401, 0.17091094707724913), (6425, 0.09079561068835709), (6663, 0.09493385571651518), (6711, 0.06283019374829278), (6850, 0.07839777762085072), (7094, 0.05680556446438203), (7180, 0.07645982421199359)]


* These are tuple that represent the documents ID and the corresponding tfidf scores where the word "pizza" occcurs.

## 2.2.2) Execute the query

### Similarity

* Now we create a function that return the cosine similarity between a query and a text (in this case we will use long descriptions), so that we can use it for next points
* The cosine similarity formula is:

\begin{equation}
\operatorname{\textbf{cosine similarity}}(\mathbf{query}, \mathbf{document})=\frac{\mathbf{query} \cdot \mathbf{document}}{|\mathbf{query} \| \mathbf{document}|}=\frac{\sum_{i=1}^N (query)_i (document)_i}{\sqrt{\sum_{i=1}^N (query)_i^2} \sqrt{\sum_{i=1}^N (document)_i^2}}
\end{equation}

where $query$ is the vector containing the tfidf scores of each word of the processed query based on the query itself and $document$ is the vector containing the tfidf score for a fixed description.

* In order to do less iterations and so speed up the function we create two different functions (sim1 and sim2) linked in the function sim_tot. We did this because sim1 can be not repeated for each description se it will be called only once. Instead sim2 is repeated for each description.

In [25]:
def sim1(query, data):
    query = pre_proces_query(query)
    tf_q = []
    q = []
    for j,word in enumerate(query):
        if word not in q:
            q.append(word)
            tf_q.append(query.count(word)/len(query))
    # idf
    idf_beta = np.zeros(len(query))
    for j,word in enumerate(query):
        count = 0
        for i in data.index:
            if word in data.loc[i]:
                count+=1
        idf_beta[j] = count
    idf = np.zeros(len(q))
    for i in range(len(q)):
        if idf_beta[i] != 0:
            idf[i] = 1 + np.log(len(data)/idf_beta[i])
        else:
            idf[i] = 0
    # tfidf_query
    tfidf_q = tf_q*idf
    return q,tfidf_q, idf 

In [26]:
def sim2(q, tfidf_q, idf,docu):
    tf_d = np.zeros(len(q))
    for j,word in enumerate(q):
        if len(docu) != 0:
            tf_d[j] = docu.count(word)/len(docu)
        else:
            tf_d[j] = 0
    tfidf_d = tf_d*idf
    product = np.dot(list(tfidf_q),list(tfidf_d))
    quer = np.sqrt(sum(tfidf_q**2))
    document =  np.sqrt(sum(tfidf_d**2))
    if quer*document != 0:
        sim = product/(quer*document)
    else:
        sim =0  
    return sim

In [27]:
def sim_tot(query, data):
    q, tfidf_q, idf = sim1(query,data)
    a = np.zeros(len(data))
    for i,d in enumerate(data):
        a[i] = (sim2(q,tfidf_q, idf,d))
    return a

* An example of similarity calculation

In [139]:
query_ex = "new york pizza"

In [141]:
sim_tot(query_ex, webdata['list_words_placeDesc'])

array([0.55413448, 0.        , 0.        , ..., 0.        , 0.55413448,
       0.        ])

### Inverted index 2

* Finally we can create a function that given a query, an integer k and the dataset, will return the first k descrptions where the query is totally conteined in and order by cosine similarity score.

In [28]:
def inverted_index2(query,k,data):
    stem_query = pre_proces_query(query)
    results =[set(inverted_idx[item]) for item in [vocabulary[word] for word in stem_query]]
    first = results[0]
    for other in results[1:]:
        first = first.intersection(other)
    docID = list(reduce(lambda x,y: x.intersection(y), results))
    val = sim_tot(query, data.loc[docID]['list_words_placeDesc'])
    dictio = dict(sorted(dict(zip(docID, val)).items(), key=operator.itemgetter(1),reverse=True))
    a = webdata.loc[dictio.keys()]
    a["similarity"] = dictio.values()
    return a[["placeName","placeDesc","placeURL","similarity"]].head(k)

* Query example

In [205]:
query_2 = "new york and pizza"
inverted_index2(query_2,4,webdata)

,placeName,placeDesc,placeURL,similarity
371,Hess Triangle,The smallest piece of property in New York Cit...,https://www.atlasobscura.com/places/hess-triangle,0.942809
6366,300 West 38th Street,Where else but in New York City would you find...,https://www.atlasobscura.com/places/300-west-3...,0.905822
5854,Taste of Persia NYC,In the glitz and polish of Manhattan’s Flatiro...,https://www.atlasobscura.com/places/taste-of-p...,0.852803
2043,The U.S. Pizza Museum,"When it comes to New York and Chicago, pizza i...",https://www.atlasobscura.com/places/us-pizza-m...,0.668437


## 3. Define a new score!

* `sim2_0` is just a variation of `sim2` from *excercise number 2*. <br>

This function also considers the **"meaninglessness"** of the words typed into the query by the user. In the previous version of the search engine we **assume** that the query is exclusively composed by words that are known to us (i.e., that are present in our vocabulary), hence the search engine must return a meaningful list of documents. However, this may not necessarily be the case! The user may mistakenly or purposely type mispelled or meaningless words that may not match any of the documents or that may decrease the **similarity** between the query and the documents.<br>

 Therefore, `sim2_0` is similar to `sim2`, but with the following differences:
  * If the `idf` is equal to 0, the word in the query is not present in any of the documents. Therefore, we define `weight` as:
  
  \begin{equation}
    Weight = \frac{(length(query) - list(idf).count(0))}{length(query)}
  \end{equation}
  
   where `list(idf).count(0)` is the number of **unknown** query words (that have `idf` equal to 0). Let's make some examples:
   * if **query = "new unknownWord1 unknownWord2"**, only one ("new") out of 3 words are known, implying that the $weight = 1/3$. 
   * If **query = "city hall station"**, all of the words of the query are known to our vocabulary, hence the $weight = 1$.
   * If **query = "unknownWord1 unknownWord2 unknownWord3"**, none of the query words are know, hence the $weight = 0$. In this case, the function will return the following message:
   
   <pre><code>
   Sorry, could not find results containing all your search terms.
   Make sure that all words are spelled correctly.
   Try different keywords.
   Try more generic keywords.
   
   </code></pre>
   
  * The *cosine similarity* will decrease (or stay invariant) according to the *weight*, for $0 \leq weight \leq 1$. 

In [29]:
def sim2_0(q, tfidf_q, idf, docu):
    tf_d = np.zeros(len(q))
    for j,word in enumerate(q):
        if len(docu) != 0:
            tf_d[j] = docu.count(word)/len(docu)
        else:
            tf_d[j] = 0
    tfidf_d = tf_d*idf
    weight = (len(q)-list(idf).count(0))/len(q)
    product = np.dot(list(tfidf_q),list(tfidf_d))
    document =  np.sqrt(sum(tfidf_d**2))
    if weight != 0:
        quer = np.sqrt(sum(tfidf_q**2)/weight)
    else:
        quer = "Sorry, could not find results containing all your search terms." + "\n" + "Make sure that all words are spelled correctly." + "\n" + "Try different keywords." + "\n" + "Try more generic keywords."
    if type(quer) != str:
        if quer*document != 0:
            sim = product/(quer*document)
        else:
            sim = 0
    else:
        sim = quer
    return sim

* `sim_tot_2` is just the updated version of `sim_tot` which uses `sim2_0` instead of `sim2`, while `sim1` stays the same.

In [30]:
def sim_tot_2(query, data):
    q, tfidf_q, idf = sim1(query,data)
    a = np.zeros(len(data))
    for i,d in enumerate(data):
        a[i] = (sim2_0(q,tfidf_q, idf,d))
    return a

### Our new score

* The function `new_score(query, data)` takes as inputs the *query* and the *data* (in our case `webdata`). <br>

* In our new score, we not only take into consideration the description `placeDesc`, but also the title `placeName` and the short description `placeShortDesc`.
  * We pre-process the title and the short and long descriptions using the `pre_proces` function.
  * Then, we calculate the *similarity* between the title and the query using `sim_tot`, the *similarity* between the short description and the query using `sim_tot`, finally we calculate the *similarity* between the long description and the query using `sim_tot_2`. We use the newly defined function `sim_tot_2` only for the long descriptions, as we don't want to excessively penalize the similarity score.
  * Finally, the `newscore` is just the *arithmetic mean* of the three similarity scores `sim_title, sim_short & sim_long`.

In [31]:
def new_score(query, data):
    stem_title = pre_proces(data,"placeName")
    stem_shortDesc = pre_proces(data,"placeShortDesc")
    stem_longDesc = pre_proces(data,"placeDesc")
    sim_title = sim_tot(query,stem_title)
    sim_short = sim_tot(query,stem_shortDesc)
    sim_long = sim_tot_2(query,stem_longDesc)
    newscore = (sim_title+sim_short+sim_long)/3
    return newscore

* Query example

In [35]:
query1 = "elinrv new york edfkjjknkj kh"
new_score(query1,webdata)

array([0.54415184, 0.        , 0.        , ..., 0.        , 0.21081851,
       0.        ])

### Inverted index 3

* Now we create a new inverted index using our new score. There will be also the possibility to insert as input an URL and the function will return the corresponding placeName, placeDesc and placeURL

In [36]:
def inverted_index3(query,k,data):
    if query in list(data['placeURL']):
        return data.loc[data['placeURL']== query,data.columns[[0,4,14]]]
    else:
        stem_query = pre_proces_query(query)
        results =[set(inverted_idx[item]) for item in [vocabulary[word] for word in stem_query]]
        first = results[0]
        for other in results[1:]:
            first = first.intersection(other)
        docID = list(reduce(lambda x,y: x.intersection(y), results))
        val = new_score(query, data)
        dictio = dict(sorted(dict(zip(docID, val)).items(), key=operator.itemgetter(1),reverse=True))
        a = webdata.loc[dictio.keys()]
        a["new_score"] = dictio.values()
        return a[["placeName","placeDesc","placeURL","new_score"]].head(k)

* Example with a normal query (using our new_score)

In [208]:
query_ind3 = "new york"
inverted_index3(query_ind3,10,webdata)

,placeName,placeDesc,placeURL,new_score
3744,Newtown Creek Nature Walk,In one of the last places in New York City whe...,https://www.atlasobscura.com/places/newtown-cr...,1.000000
1416,Hamilton Grange,Built in the pastoral expanses of colonial Har...,https://www.atlasobscura.com/places/hamilton-g...,0.996916
0,City Hall Station,The first New York City subway was built and o...,https://www.atlasobscura.com/places/city-hall-...,0.666667
2202,Les Jardins de Quatre-Vents,Francis Higginson Cabot loved gardens more tha...,https://www.atlasobscura.com/places/les-jardin...,0.666667
7039,Susan B. Anthony's Grave,At the historic and beautiful Mt. Hope Cemeter...,https://www.atlasobscura.com/places/susan-b-an...,0.666667
5942,Site of the Beach Pneumatic Subway,Alfred Beach built this pneumatic tunnel to sh...,https://www.atlasobscura.com/places/beach-pneu...,0.666667
4057,Venetian Room,This gem of a room at the French Embassy in Ne...,https://www.atlasobscura.com/places/venetian-room,0.660379
1217,Whirligig Park (Formerly Acid Park),"According to urban legend in Wilson, North Car...",https://www.atlasobscura.com/places/whirligig-...,0.660379
752,Yonah Schimmel's Knish Bakery,As much of New York’s old Lower East Side disa...,https://www.atlasobscura.com/places/yonah-schi...,0.656535
4061,The Sugar House Prison Window,Hidden away on a wall of the New York City Pol...,https://www.atlasobscura.com/places/the-sugar-...,0.656535


* Example with the same query but using the cosine similarity

In [209]:
inverted_index2(query_ind3,10,webdata)

,placeName,placeDesc,placeURL,similarity
18,Roosevelt Island Smallpox Hospital Ruins,Few diseases have had a greater impact on the ...,https://www.atlasobscura.com/places/roosevelt-...,1.0
6162,Coney Island Creek,"At nearly 10 acres, Coney Island Creek is a si...",https://www.atlasobscura.com/places/coney-isla...,1.0
6166,Castle Craig,This lone tower pokes above the rocks within a...,https://www.atlasobscura.com/places/castle-craig,1.0
4128,Assassin's End,"On the evening of April 14th, 1865, famed acto...",https://www.atlasobscura.com/places/assassins-end,1.0
4134,Jackson Sanatorium,"In its lifetime, the Jackson Sanatorium has ha...",https://www.atlasobscura.com/places/jackson-sa...,1.0
2087,The Underground Home,"In the 1960s, the Cuban Missile Crisis posed a...",https://www.atlasobscura.com/places/the-underg...,1.0
57,Yayoi Kusama Firefly Infinity Mirror Room,The 89-year-old Japanese avant-garde artist Ya...,https://www.atlasobscura.com/places/yayoi-kusa...,1.0
69,Track 61,Unlike other “abandoned” train stations in the...,https://www.atlasobscura.com/places/track-61,1.0
71,The SeaGlass Carousel,The SeaGlass Carousel in New York City’s Batte...,https://www.atlasobscura.com/places/the-seagla...,1.0
75,The 'Ghostbusters' Firehouse,As you walk down North Moore Street in lower M...,https://www.atlasobscura.com/places/ghostbuste...,1.0


* As we can see our new score will be more precise ordering the place because it will analyze also the placeName and the placeShortdesc for each place.

* As we said, if we give in input an URL it will return the right row of the dataset:

In [89]:
inverted_index3("https://www.atlasobscura.com/places/roosevelt-island-smallpox-hospital-ruins",10,webdata)

,placeName,placeDesc,placeURL
18,Roosevelt Island Smallpox Hospital Ruins,Few diseases have had a greater impact on the ...,https://www.atlasobscura.com/places/roosevelt-...


## 4. Visualizing the most relevant places

* Now let visualize on an earth map the locations of the k place more related (with our new_score) to a specific query

In [42]:
query_4 = "new york"

* In order to not change our previusly function output, we do a merge between the original dataframe and the one with only placeName, placeDesc and placeURL of first k ordered places. This is because we will need latitude and longitude for each of these places.

In [43]:
df = inverted_index3(query_4,10,webdata)

In [44]:
df_k = pd.merge(df,webdata,on=["placeName","placeDesc","placeURL"])

In [ ]:
map_new=folium.Map()
list_tags = list(df_k['placeTags'])
list_lon = list(df_k['placeLong'])
list_lat = list(df_k['placeAlt'])
for i in range(len(list_lon)):
    map_new.add_child(folium.Marker(location=[list_lat[i],list_lon[i]], popup=list_tags[i],icon=folium.Icon(color='green')))
map_new  

* This is the final representation 

<img src="map_point_4_0.jpg"  width=80% height=80%> <img src="map_point_4.jpg"  width=80% height=80%>

* These are two images of the same map with different zoom levels for the example query

# 7 Teoretical question

An imaginary university wants to create a ranking list of the applicants for positions to study the Master of Data Science there.Students are then admitted based on how well they perform on these exams, they should be ranked in the list based on their average points in descending order, if two students have the same average punctuation, they should be sorted in ascending order using their first and last names.
University will give you the students' information in 'ApplicantsInfo.txt' and you should provide them with the ranking list in another .txt file and name it as 'RankingList.txt' .

## 7.1

Try solving the problem mentioned above using three different sorting algorithms (do not use any MapReduce algorithm). (Note: Built-in Python functions (like .mean, .sort, etc.) are not allowed to be used. You must implement the algorithms from scratch).

### Auxiliary function

#### read ApplicantsInfo.txt: students and averages lists
The below function reads line by line the file "ApplicantsInfo.txt" and creates two lists the firt containg name and surname of the students and in the other one stores the exams'average, these lists will be the starting point for the sorting algorithms.

In [2]:
def ApplicantsInfo(): 
 with open(r"C:\Users\wiwis\OneDrive\Desktop\ADM\3°HM\ApplicantsInfo.txt",'r') as f:
   n,m=list(map(int,f.readline().split()))
        
   current_line_list=f.readline().split()
   averages_list=["{:.2f}".format(sum(list(map(int,current_line_list[2:])))/m)]
   students_list=[current_line_list[0]+' '+current_line_list[1]]
    
   for i in range(1,n): 
     current_line_list=f.readline().split()
     averages_list.append("{:.2f}".format(sum(list(map(int,current_line_list[2:])))/m))   
     students_list.append(current_line_list[0]+' '+current_line_list[1])  
   
 return n,m,averages_list,students_list

In [3]:
n,m,averages_list_unsorted,students_list_unsorted=ApplicantsInfo() 

#### write RankingList.txt 
The function RankingList creates the file 'RankingList.txt' using sorted lists that will be returned by  the sorting algorithms.

In [4]:
def RankingList(averages_list, students_list, sorting):
  with open(r"C:\Users\wiwis\OneDrive\Desktop\ADM\3°HM\RankingList"+sorting+".txt",'w') as g:
          g.write(students_list[0]+' '+str(averages_list[0])+'\n') 
          for i in range(1,n-1):                                          
              g.write(students_list[i]+' '+str(averages_list[i])+'\n')  
          g.write(students_list[n-1]+' '+str(averages_list[n-1]))  

### Sorting algorithm

#### sorting by BINARY SEARCH

In [5]:
def binarySort(averages_list_unsorted,students_list_unsorted,n,m):  
   averages_list=[averages_list_unsorted[0]]
   students_list=[students_list_unsorted[0]]
  

   for i in range(1,n):
     current_average=averages_list_unsorted[i]
     current_student=students_list_unsorted[i]
     
     
    
     if current_average >= averages_list[0]:
        if current_average > averages_list[0]:
          averages_list.insert(0, current_average )
          students_list.insert(0,current_student)
        else:
          j=0
          current_length=len(averages_list)  
          while j+1<current_length and averages_list[j] == current_average and current_student > students_list[j]     :
                j+=1
          averages_list.insert(j, current_average )
          students_list.insert(j,current_student)   
         
     elif current_average <= averages_list[-1]:
        if current_average < averages_list[-1]:
          averages_list.append(current_average )
          students_list.append(current_student)
        else:
          j=len(averages_list)
          while j-1>=0 and averages_list[j-1] == current_average and  current_student < students_list[j-1]    :
                j+=(-1)
          averages_list.insert(j, current_average)
          students_list.insert(j,current_student)
     else:    
        first = 0
        last = len(averages_list)-1
        found=False
        while first<last and not(found):
              j = (first + last)//2  #midpoint
              if current_average == averages_list[j]:
                found=True
                while j-1>=0 and averages_list[j] == current_average and current_student < students_list[j]    :
                      j+=(-1)
                j+=1        
                current_length=len(averages_list)         
                while j+1<current_length and averages_list[j] == current_average and current_student > students_list[j]    :
                      j+=1        
              else:  
                if current_average > averages_list[j]:
                   last = j-1
                else:
                   first = j+1
        
        if averages_list[j]> current_average:
           current_length=len(averages_list)         
           while j+1<current_length and  current_average == averages_list[j+1] and  current_student > students_list[j+1]  :
                      j+=1 
           averages_list.insert(j+1, current_average )
           students_list.insert(j+1,current_student)
        elif averages_list[j]<current_average:
           while  j-1>=0 and averages_list[j-1]==current_average and current_student < students_list[j-1]:
                      j+=(-1)
           averages_list.insert(j, current_average )
           students_list.insert(j,current_student)
        else:
           averages_list.insert(j, current_average )
           students_list.insert(j,current_student)
            
   return  averages_list, students_list    

In [6]:
averages_list_sort, students_list_sort = binarySort(averages_list_unsorted,students_list_unsorted,n,m)

In [7]:
RankingList(averages_list_sort , students_list_sort, 'Binarysearch')
#print('The algorithm perform in '+ str((b-a)[0])+' hours '+ str((b-a)[1])+' mins '+ str(((b-a)[2]+60)%60)+' sec')

#### sorting by MERGESORT

In [5]:
def merge(averages_list, students_list, l, m, r):  # let n_merge=(r-l)
    n1 = m - l + 1        
    n2 = r - m               
    
    #2 op(addition)
    
    L_av= [0]*n1
    L_st= [0]*n1 
    R_av = [0]*n2 
    R_st = [0]*n2 
    
    #2*n_merge op(allocation)
    
    for i in range(0, n1):
        L_av[i] = averages_list[l + i]
        L_st[i] = students_list[l + i]
    for j in range(0, n2):
        R_av[j] = averages_list[m + 1 + j]
        R_st[j] = students_list[m + 1 + j]
     
    
    #2*n_merge op(allocation)
    
    i = 0
    j = 0
    k = l
    
    #3 op(allocation)
    
    while i < n1 and j < n2:
        if L_av[i] > R_av[j] or (L_av[i] == R_av[j] and L_st[i]<=R_st[j]):
            averages_list[k] = L_av[i]
            students_list[k] = L_st[i]
            i += 1     
        else:        
            averages_list[k] = R_av[j]
            students_list[k] = R_st[j]
            j += 1            
        k += 1
    
    # 2*(k) op(boolean check) (while and if) + 2*(k) op(allocation) where min(n1,n2)<= k <=n
    
    while i < n1:    
        averages_list[k] = L_av[i]
        students_list[k] = L_st[i]
        i += 1
        k += 1

    while j < n2:    
        averages_list[k] = R_av[j]
        students_list[k] = R_st[j]
        j += 1
        k += 1
        
    # (n_merge-k+1) op(boolean check) + (n_merge-k) op(allocation)
    
    return averages_list,students_list

def mergeSort(averages_list,students_list, l, r): #let n=(r-l)
    if l < r:
        m=(l+r)//2   
       
        mergeSort(averages_list, students_list, l, m)    #T(m-l)~T(n/2)
        mergeSort(averages_list, students_list, m+1, r)  #T(r-m+1)~T(n/2)
        merge(averages_list, students_list, l, m, r)     #T_merge(n)
    return averages_list,students_list 

In [6]:
averages_list_sort =  averages_list_unsorted.copy()
students_list_sort =  students_list_unsorted.copy()
  
averages_list_sort,students_list_sort = mergeSort(averages_list_sort,students_list_sort, 0, len(averages_list_sort)-1)

In [7]:
RankingList(averages_list_sort , students_list_sort, 'Merge')

#### sorting by QUICKSORT

In [11]:
def partition(averages_list,students_list, l, r):    #let n_partition=(r-l)                                                                                             
                                                                                                   
    pivot = averages_list[r]                                                                                                  
    i = l - 1  
    
    #2 op(allocation)
    
    for j in range(l, r):                                                                                                                                                                                     
        if averages_list[j] > pivot or (averages_list[j] == pivot and students_list[j]< students_list[r]):                                                                                                                                                                                     
            i = i + 1                                                                                                                                                                                      
            (averages_list[i], averages_list[j]) = (averages_list[j], averages_list[i])                                                                                        
            (students_list[i], students_list[j]) = (students_list[j], students_list[i])                                                                                                                                                                                    
    
    # 2*n_partition op(boolean check) +  3*(n_partition-k) op(allocation)     where 0 <= k <=n
    
    (averages_list[i + 1], averages_list[r]) = (averages_list[r], averages_list[i + 1])                                                                                                                                   
    (students_list[i + 1], students_list[r]) = (students_list[r], students_list[i + 1])    
    
    #2 op(allocation)
    
    return i + 1                                                                                                                                                                                     
                                                                                                  
                                                                                                                                                                                      
def quickSort(averages_list,students_list, l, r):   #let n=(r-l)
    if l < r:                                                                                                                                                                                     
        pi = partition(averages_list,students_list, l, r)        #T_partition(n)                                                                                                                                                                             
        quickSort(averages_list,students_list, l, pi - 1)        #T((pi-1)-l)
        quickSort(averages_list,students_list, pi + 1,r)         #T(r-(pi+1))                                                                                                                                                                            
                                                                                                                                                                                                                                              
    return averages_list,students_list                                                                                                                                    

In [12]:
averages_list_sort =  averages_list_unsorted.copy()
students_list_sort =  students_list_unsorted.copy()

averages_list_sort,students_list_sort = quickSort(averages_list_sort,students_list_sort, 0, len(students_list_sort)-1)

In [13]:
RankingList(averages_list_sort , students_list_sort, 'Quick')

## 7.2

 What is the time complexity of each algorithm you have used?

* ###  complexity **mergeSort** : 
<br>
Adding up the number of all operations in the code of the procedure "merge" we have:
<br><br>
$
\begin{align*}
T_{\text{merge}}(n_{\text{merge}})=2+2n_{\text{merge}} + 2n_{\text{merge}} +3 + 2k +2k + (n_{\text{merge}}-k+1) + (n_{\text{merge}}-k)
\end{align*}
$
<br><br>
where  $\quad 0 \leq min\{n_1,n_2\} \leq k \leq n \quad$ with $n_1$ and $n_2$ as defined in the code, so
<br><br> 
$ 
\begin{align*}
T_{\text{merge}}(n_{\text{merge}}) = 5+6n_{\text{merge}} +2k \qquad \implies \qquad 5+6n_{\text{merge}} \leq T_{\text{merge}}(n_{\text{merge}}) \leq 5+8n_{\text{merge}} \quad (\ast)
\end{align*}
$
<br><br>
the global complexity $T(n)$ of the algorithm as shown in the code is:
<br><br>
$
\begin{align*}
T(n) \quad = \quad 2 T\left( \frac{n}{2} \right) +T_{\text{merge}}(n) \quad = \quad 2\left( 2 T\left( \frac{n}{2} \cdot \frac{1}{2}\right)+T_{\text{merge}}\left( \frac{n}{2} \right) \right)  +T_{\text{merge}}\left( n \right)
\end{align*}
$
<br><br>
$
\begin{align*}
= \quad \dots \quad = \quad 2^iT\left( \frac{n}{2^i} \right) +\sum_{j=1}^i 2^{j-1} T_{\text{merge}}\left( \frac{n}{2^{j-1}} \right)
\end{align*}
$
<br><br>
$
\begin{align*}
\implies T(n) \quad = \quad 2^{\log_2(n)} T\left( \frac{n}{2^{\log_2(n)}} \right) +\underbrace{\sum_{j=1}^{\log_2(n)} 2^{j-1}T_{\text{merge}}\left( \frac{n}{2^{j-1}} \right)}_{A}
\end{align*}
$
<br><br>
focusing on $A$ ...
<br><br>
$
\begin{align*}
\sum_{j=1}^{\log_2(n)} 2^{j-1}T_{\text{merge}}\left( \frac{n}{2^{j-1}} \right) \quad \leq \quad \sum_{j=1}^{\log_2(n)} 2^{j-1}\left( 5 + 8 \cdot \frac{n}{2^{j-1}} \right) \quad = \quad 8n \log_2(n) +5\sum_{j=1}^{\log_2(n)} 2^{j-1}
\end{align*}
$
<br><br>
$
\begin{align*}
\sum_{j=1}^{\log_2(n)} 2^{j-1} \quad = \quad \sum_{j=0}^{\log_2(n)-1} 2^{j} \quad = \quad \frac{1-2^{\left( \log_2(n)-1 \right) +1}}{1-2} \quad = \quad n -1
\end{align*}
$
<br><br>
in conclusion
<br><br>
$
\begin{align*}
\implies T(n) \quad \leq \quad nT(1) +  8n \log_2(n) +5(n -1) \qquad \text{ and similarly using $(\ast)$} \quad T(n) \quad \geq \quad nT(1) +  6n \log_2(n) +5(n -1)
\end{align*}
$
<br><br>
Since $\quad T(1)=1 \quad $ results $\quad T(n)= \Theta\left( 6n \log_2(n)+6n \right) \quad $ or equivalent $\quad T(n)= \Theta\left( n \log_2(n)\right) \quad $.

* ###  complexity **quickSort** :

<br>
Adding up the number of all operations in the code of the procedure "partition" we have:
<br><br>
$
\begin{align*}
T_{\text{partition}}(n_{\text{partition}})= 2+2 n_{\text{partition}} +3\left(n_{\text{partition}} -k \right) +2
\end{align*}
$
<br><br>
where  $\quad 0  \leq k \leq n \quad$, so
<br><br> 
$ 
\begin{align*}
T_{\text{partition}}(n_{\text{partition}}) = 4+5n_{\text{partition}} -3k \qquad \implies \qquad 4+2n_{\text{merge}} \leq T_{\text{partition}}(n_{\text{partition}}) \leq 4+5n_{\text{merge}} \quad (\ast)
\end{align*}
$
<br><br>
The global complexity $T(n)$ of the algorithm as shown in the code is:
<br><br>
$
\begin{align*}
T(n) \quad = \quad  T\left( (pi-1)-l \right) + T\left( r-(pi+1) \right)  +T_{\text{partition}}(n) 
\end{align*}
$
<br><br>
with $l$,$r$ and $pi$ as defined in the code. <br>
Now we can interpret the computational complexity of the algorithm as a random variable $T(n,pi)$ dependent on the position of the pivot $pi$, whose probability of occupying any position $j$ after the partition procedure can be assumed to be $P(pi=j)=1/n$; with these assumptions we can approximate the average complexity with its expected value.
<br><br>
$
\begin{align*}
T(n) \quad = \quad \mathbb{E}(T(n,pi)) \quad = \sum_{j=0}^{n-1} P(pi=j) T(n,pi=j) \quad  \implies T(n) \quad = \quad \sum_{j=0}^{n-1} \frac{1}{n} \big( T \left( j \right) + T\left( n-(j+1) \right)  +T_{\text{partition}}(n) \big)
\end{align*}
$
<br><br>
we observe that $T \left( j \right)$ and $T\left( n-(j+1) \right)$ generate the same terms, so
<br><br>
$
\begin{align*}
T(n) \quad = \quad T_{\text{partition}}(n) + \frac{2}{n}\sum_{j=0}^{n-1} T \left( j \right) \qquad
\implies \quad nT(n) \quad = \quad nT_{\text{partition}}(n) + 2\sum_{j=0}^{n-1} T \left( j \right) \qquad (a)
\end{align*}
$
<br><br>
similarly
<br><br>
$
\begin{align*}
(n-1)T(n-1) \quad = \quad (n-1)T_{\text{partition}}(n-1) + 2\sum_{j=0}^{n-2} T \left( j \right) \qquad  (b)
\end{align*}
$
<br><br>
subtracting $(b)$ from $(a)$
<br><br>
$
\begin{align*}
nT(n) -(n-1)T(n-1) \quad = \quad nT_{\text{partition}}(n) - (n-1)T_{\text{partition}}(n-1) + 2 T \left( n-1 \right) 
\end{align*}
$
<br><br>
$
\begin{align*}
\leq \quad n(4+5n) - (n-1)(4+5(n-1)) + 2 T \left( n-1 \right) \quad = \quad \dots \quad = \quad 10n +9 + 2 T \left( n-1 \right) \qquad \implies \quad nT(n) \leq (n+1)T(n-1) + 10n +9
\end{align*}
$
<br><br>
$
\begin{align*}
\implies \frac{T(n)}{(n+1)} \leq \frac{T(n-1)}{n} + \frac{10}{(n+1)} +\underbrace{\frac{9}{n(n+1)}}_{\text{negligible because $\sum \frac{1}{n^2}$ converges}} \leq \left(  \frac{T(n-2)}{n-1} + \frac{10}{n} \right) + \frac{10}{(n-1)} \quad \leq \quad \dots \quad \leq \quad \frac{T(1)}{2} + 10 \underbrace{ \sum_{j=1}^n \frac{1}{j+1} }_{B}
\end{align*}
$
<br><br>
it is possible to show that for n tending to infinity ( reference 1) $B$ tends to $\quad \ln n + \gamma \quad $ where $\quad \gamma=0.577 \dots \quad$ is the Eulero-Mascheroni's costant, since $T(1)$ is equal to 1  this two condition implies 
<br><br>
$
\begin{align*}
T(n) \quad \leq \quad \frac{n+1}{2} + 10(n+1) \ln (n) +(n+1)\gamma \qquad \implies \qquad T(n) =O(n \log(n))
\end{align*}
$
<br><br>
To find a lowerbound of the complexity we know that $\quad T(n)=\Omega(T_{\text{best}}(n)) \quad $ and since the best instance for the algoritm correspond to the case when for each recursive call of the algorithm the pivot occurs always in the midpoint position, we have under this assumptions, $\quad T_{\text{best}}(n)= 2  T_{\text{best}}(n/2) + T_{\text{partition}}(n) \quad $ but this setting is equivalent to the previous one regarding the mergeSort, because $\quad T_{\text{partition}}(n)=\Theta(n)=T_{\text{merge}}(n) \quad$, so in conclusion $\quad T_{\text{best}}(n)= T_{\text{mergeSort}}(n)=\Theta(n \log(n))\quad $ and $\quad T(n)=\Theta(n \log(n)) \quad$

#### References

1. [https://mate.unipv.it/gilardi/WEBGG/PSPDF/eulero-masch.pdf](https://mate.unipv.it/gilardi/WEBGG/PSPDF/eulero-masch.pdf)

## 7.3 

Evaluate the time taken for each of your implementations to answer the query stored in the ApplicantsInfo.txt file and visualize them.
<br>
We estimate the time taken by the three algorithms to compute the scores' averages and sort the students in the order demand from the question 100 times.

In [14]:
start_time_binary=datetime.now()

for i in range(100):
 averages_list_sort, students_list_sort = binarySort(averages_list_unsorted,students_list_unsorted,n,m)

end_time_binary=datetime.now()

In [15]:
start_time_merge=datetime.now()

for i in range(100):
  averages_list_sort =  averages_list_unsorted.copy()
  students_list_sort =  students_list_unsorted.copy()
  
  averages_list_sort,students_list_sort = mergeSort(averages_list_sort,students_list_sort, 0, len(averages_list_sort)-1)
  
end_time_merge=datetime.now()

In [16]:
start_time_quick=datetime.now()

for i in range(100):
  averages_list_sort =  averages_list_unsorted.copy()
  students_list_sort =  students_list_unsorted.copy()
    
  averages_list_sort,students_list_sort = quickSort(averages_list_sort,students_list_sort, 0, len(students_list_sort)-1)

end_time_quick=datetime.now()

Here the result:

In [20]:
time_binary=str(end_time_binary-start_time_binary).split(':')
time_merge=str(end_time_merge-start_time_merge).split(':')
time_quick =str(end_time_quick-start_time_quick).split(':')

In [22]:
print('the algorithm binarySort takes '+time_binary[1]+' mins and '+time_binary[2][:2]+' sec')
print('the algorithm mergeSort takes '+time_merge[1]+' mins and '+time_merge[2][:2]+' sec')
print('the algorithm quickSort takes '+time_quick[1]+' mins and '+time_quick[2][:2]+' sec')

the algorithm binarySort takes 08 mins and 04 sec
the algorithm mergeSort takes 01 mins and 44 sec
the algorithm quickSort takes 00 mins and 50 sec


## 7.4

What is the most optimal algorithm, in your opinion, and why?

<br>

The quickSort algorithm takes about half as long as the mergeSort to run, it is therefore the fastest to satisfy the requirement of ordering the students' scores, the binarySort on the other hand observing the conclusions in 7. 3 is very slow, compared to the other two-algorithms it has, however, the merit of not using a recursive procedure in fact such an implementation, after making a few attempts, turns out to be inefficient when the input list is sorted opposite to the desired one, if so in fact the excessive number of recursive calls leads to the forced kernel interruption (at least using computers with ordinary characteristics), in conclusion since the list "averages_list_unsorted" given as input does not present this problem, it is legitimate to suppose that this list of numeric values of the students' averages does not present an order,  in the specific case of question 7 the quickSort algorithm turns out to be the optimal one. 